In [1]:
%matplotlib inline

In [2]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)#禁用素有训练
model=inception_v3.InceptionV3(
    weights='imagenet',
    include_top=False
)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
layer_contributions={
    'mixed2':0.3,
    'mixed3':2.5,
    'mixed4':1.5,
    'mixed5':1.4
}

In [4]:
layer_dict = dict(
    [
        (layer.name, layer) for layer in model.layers
    ]
)
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scailing = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2:-2, 2:-2, :]))


In [5]:
dream=model.input
grads=K.gradients(loss,dream)[0]
grads/=K.maximum(K.mean(K.abs(grads)),1e-7)

outputs=[loss,grads]
fetch_loss_and_grads=K.function([dream],outputs)

In [6]:
def eval_loss_and_grads(x):
    outs=fetch_loss_and_grads([x])
    loss_value=outs[0]
    grad_values=outs[1]
    return loss_value,grad_values
def gradient_ascent(x,iterations,step,max_loss=None):
    for i in range(iterations):
        loss_value,grad_values=eval_loss_and_grads(x)
        if max_loss is not None and loss_value>max_loss:
            break
        print("loss valuse at ",i," is" , loss_value)
        x+=step*grad_values
    return x


In [7]:
import scipy
from keras.preprocessing import image
def resize_img(img,size):
    img=np.copy(img)
    factors=(
        1,
        float(size[0])/img.shape[1],
        float(size[1])/img.shape[2],
        1
    )
    return scipy.ndimage.zoom(img,factors,order=1)

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x=x.reshape((3,x.shape[2],x.shape[3]))
        x=x.transpose((1,2,0))
    else:
        x=x.reshape((x.shape[1],x.shape[2],3))
    x/=2
    x+=0.5
    x*=255
    x=np.clip(x,0,255).astype('uint8')
    return x

In [8]:
def save_img(img,fname):
    pil_img=deprocess_image(np.copy(img))
    scipy.misc.imsave(fname,pil_img)

In [9]:

def preprocess_image(image_path):
    # Util函数:调整图片大小并将图片格式化为适当的张量。
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [10]:
import numpy as np
# 使用超参数获得新效果
step = 0.01  # 梯度上升步长
num_octave = 3  # 运行梯度上升的比例数
octave_scale = 1.4  # 尺度之间的尺寸比
iterations = 20  # 每个比例的上升步数
 
#如果损失大于10，我将中断梯度上升过程，以避免ugly artifacts
max_loss = 10.
 
# 将其填充到要使用的图像的路径
base_image_path = 'E:\个人练习生\skitlearn\\timg (1).jpg'
 
# 将图像加载到Numpy数组中
img = preprocess_image(base_image_path)
 
# 准备一个形状元组列表，定义了我们将运行梯度上升的不同尺度
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
 
# 反转shape列表，使它们按顺序递增
successive_shapes = successive_shapes[::-1]
 
# 将图像的Numpy数组调整为最小比例
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])
 
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
 
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
 
save_img(img, fname='final_dream.png')
 
 
 
 
from matplotlib import pyplot as plt
 
plt.imshow(deprocess_image(np.copy(img)))
plt.show()


Processing image shape (459, 816)


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]